In [1]:
# %cd ../../
%load_ext autoreload
%autoreload 2
from dataset.custom_dataset import PairedInstructionDataset

In [2]:
from transformer_lens import HookedTransformer
import torch
import json
from transformers import AutoTokenizer

In [3]:
with open('localizations/eap/eap_sports/sports_data.json') as f:
    data = json.load(f)

corr_sub_map = data['corr_sub_map']
clean_sub_map = data['clean_sub_map']

model = HookedTransformer.from_pretrained(
    'pythia-2.8b',
    device='cuda',
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    default_padding_side="left"
)
tokenizer=model.tokenizer

model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

def tokenize_instructions(tokenizer, instructions):
    # Use this to put the text into INST tokens or add a system prompt
    return tokenizer(
        instructions,
        padding=True,
        truncation=False,
        return_tensors="pt",
        # padding_side="left",
    ).input_ids

dataset = PairedInstructionDataset(
    N=1000,
    instruction_templates=data['instruction_templates'],
    harmful_substitution_map=corr_sub_map,
    harmless_substitution_map=clean_sub_map,
    tokenizer=tokenizer,
    tokenize_instructions=tokenize_instructions, 
    device='cuda'
)

corr_dataset = dataset.harmful_dataset
clean_dataset = dataset.harmless_dataset

Loaded pretrained model pythia-2.8b into HookedTransformer


In [4]:
print(clean_dataset.str_prompts)
print(corr_dataset.str_prompts)

['<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Austin Rivers plays the sport of', '<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Adam Dunn plays the sport of', '<|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Raymond Felton plays the sport of', '<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Jordan Bell plays the sport of', '<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Marvin Harrison plays the sport of', '<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Bill Self plays the sport of', '<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Matt Adams plays the sport of', '<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tige

In [5]:
with open('localizations/eap/eap_sports/sport_answers.json') as f:
    answers = json.load(f)
    player_to_sport_tok = {}
    set_of_sports = set(['football', 'baseball', 'basketball'])
    sport_to_tok = {}
    for sport in set_of_sports:
        sport_to_tok[sport] = tuple(tokenizer(' ' + sport).input_ids)

    for player, sport in zip(answers['players'], answers['sports']):
        wrong_sports = set_of_sports - {sport}
        wrong_sport_toks = [sport_to_tok[wrong_sport] for wrong_sport in wrong_sports]
        player_tuple = tuple(tokenizer(' ' + player).input_ids)
        player_to_sport_tok[player_tuple] = (sport_to_tok[sport], wrong_sport_toks)

In [6]:
clean_tok_answers = []
clean_tok_wrong_answers = []

for i in range(len(clean_dataset.deltas)):
    start = clean_dataset.deltas[i]["{player}"].start
    end = clean_dataset.deltas[i]["{player}"].stop
    correct_sport_tok, wrong_sports_toks = player_to_sport_tok[tuple(clean_dataset.toks[i, start:end].tolist())]
    clean_tok_answers.append(
        torch.tensor(correct_sport_tok)
    )
    clean_tok_wrong_answers.append(
        torch.tensor(wrong_sports_toks)
    )
clean_tok_answers = torch.tensor(clean_tok_answers, device='cuda:0')
clean_tok_wrong_answers = torch.stack(clean_tok_wrong_answers).to(device='cuda:0')
print(clean_tok_answers)
print(clean_tok_wrong_answers)

tensor([14648, 14623, 14648, 14648,  5842, 14648, 14623, 14623, 14648, 14648,
        14623, 14648, 14623,  5842, 14648,  5842, 14623,  5842,  5842,  5842,
         5842, 14648,  5842,  5842, 14648,  5842, 14623, 14648,  5842,  5842,
         5842, 14623, 14648,  5842,  5842,  5842, 14648, 14648,  5842, 14648,
        14648,  5842, 14623,  5842,  5842, 14623,  5842, 14648,  5842,  5842,
         5842, 14648, 14623,  5842,  5842,  5842, 14623,  5842,  5842,  5842,
        14648, 14623,  5842, 14648, 14623, 14648, 14623, 14623, 14623, 14648,
        14648, 14648,  5842,  5842,  5842, 14648,  5842,  5842,  5842, 14648,
        14623,  5842, 14623,  5842,  5842, 14648,  5842, 14648, 14623,  5842,
         5842,  5842,  5842, 14648, 14648,  5842, 14623, 14648, 14623,  5842,
        14648,  5842, 14648,  5842, 14623,  5842, 14648, 14648,  5842,  5842,
        14648,  5842,  5842,  5842,  5842,  5842, 14648,  5842, 14623, 14648,
         5842, 14648,  5842, 14623, 14623,  5842, 14623,  5842, 

In [7]:
from localizations.eap.eap_wrapper import EAP
from tasks.inference_utils import get_final_logits

def ave_logit_diff(
    logits,
    clean_answers,
    wrong_answers,
    per_prompt: bool = False,
    verbose=False
):
    '''
        Return average logit difference between correct and incorrect answers
    '''
    if verbose:
        print(logits.shape)
        last_logits = logits[:, -1, :]

        # Get the top 10 logits and their indices for the last sequence position across the batch
        top_logits, top_indices = torch.topk(last_logits, 10, dim=-1)

        # # Decode the top indices to tokens for each item in the batch
        top_tokens = [tokenizer.decode(indices.tolist()) for indices in top_indices]

        print(f"{top_tokens=}\n{top_logits=}")
    
    print(f"{logits.shape}, {clean_answers.shape}, {wrong_answers.shape}")
    logits_correct = logits[list(range(logits.size(0))), -1, clean_answers]
    logits_incorrect = logits[list(range(logits.size(0))), -1, wrong_answers].mean(dim=1)
    logit_diff = logits_correct - logits_incorrect
    return logit_diff if per_prompt else logit_diff.mean()

with torch.no_grad():
    clean_logits = model(clean_dataset.toks)
    corrupt_logits = model(corr_dataset.toks)
    clean_logit_diff = ave_logit_diff(
        clean_logits, 
        clean_answers=clean_tok_answers, 
        wrong_answers=clean_tok_wrong_answers
    ).item()
    corrupt_logit_diff = ave_logit_diff(
        corrupt_logits, 
        clean_answers=clean_tok_answers, 
        wrong_answers=clean_tok_wrong_answers
    ).item()

def refusals_metric(
    logits,
    clean_answers=clean_tok_answers,
    wrong_answers=clean_tok_wrong_answers,
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
 ):
    patched_logit_diff = ave_logit_diff(
        logits,
        clean_answers,
        wrong_answers,
    )
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

# Get clean and corrupt logit differences
with torch.no_grad():
    clean_metric = refusals_metric(clean_logits, corrupted_logit_diff=corrupt_logit_diff, clean_logit_diff=clean_logit_diff)
    corrupt_metric = refusals_metric(corrupt_logits, corrupted_logit_diff=corrupt_logit_diff, clean_logit_diff=clean_logit_diff)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

OpenAI API key not found, will not be able to run evaluations on HPSAQ Task
OpenAI API key not found, will not be able to run evaluations on HPSAQ Task
torch.Size([1000, 22, 50304]), torch.Size([1000]), torch.Size([1000, 2, 1])
torch.Size([1000, 22, 50304]), torch.Size([1000]), torch.Size([1000, 2, 1])
torch.Size([1000, 22, 50304]), torch.Size([1000]), torch.Size([1000, 2, 1])
torch.Size([1000, 22, 50304]), torch.Size([1000]), torch.Size([1000, 2, 1])
Clean direction: 2.979696750640869, Corrupt direction: -0.09569133073091507
Clean metric: 1.0, Corrupt metric: 0.0


In [8]:
tokenizer.decode(clean_dataset.toks[0])

'<|endoftext|><|endoftext|><|endoftext|><|endoftext|>Fact: Tiger Woods plays the sport of golf\nFact: Austin Rivers plays the sport of'

In [9]:
from tasks import SportsTask
sports_task = SportsTask(32, tokenizer, device='cuda:0')
print(sports_task.get_test_accuracy(model, check_all_logits=True))
print(sports_task.get_logit_diff(model))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0.0625
tensor(2.5527, device='cuda:0')


In [10]:

# %%

model.reset_hooks()

#%%
from localizations.eap.eap_wrapper import EAP

graph = EAP(
    model,
    clean_dataset.toks,
    corr_dataset.toks,
    refusals_metric,
    upstream_nodes=["mlp", "head"],
    downstream_nodes=["mlp", "head"],
    batch_size=10,
    clean_answers=clean_tok_answers,
    wrong_answers=clean_tok_wrong_answers,
)

# %%

top_edges = graph.top_edges(n=100, abs_scores=True)
print(top_edges)

# %%

# graph.show(threshold=0.01, abs_scores=True, fname="eap_subgraph_bs=100.png")

Saving activations requires 0.0101 GB of memory per token


  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([10, 22, 50304]), torch.Size([10]), torch.Size([10, 2, 1])


  1%|          | 1/100 [00:08<14:33,  8.82s/it]

torch.Size([10, 22, 50304]), torch.Size([10]), torch.Size([10, 2, 1])


...

torch.Size([10, 22, 50304]), torch.Size([10]), torch.Size([10, 2, 1])


 99%|█████████▉| 99/100 [14:52<00:08,  8.78s/it]

torch.Size([10, 22, 50304]), torch.Size([10]), torch.Size([10, 2, 1])


100%|██████████| 100/100 [15:02<00:00,  9.02s/it]


[('head.0.2', 'mlp.0', 0.009443754330277443), ('head.0.14', 'mlp.0', 0.006188404746353626), ('mlp.6', 'mlp.8', 0.00573932658880949), ('mlp.0', 'mlp.2', 0.005653967149555683), ('head.16.20', 'mlp.16', -0.005345507059246302), ('head.16.20', 'head.17.30.v', 0.005315450485795736), ('mlp.0', 'head.1.16.k', -0.005109565332531929), ('head.14.14', 'mlp.15', -0.004921694286167622), ('mlp.15', 'head.16.20.k', -0.004847021773457527), ('mlp.6', 'mlp.15', -0.004780464340001345), ('mlp.10', 'head.16.20.k', 0.004766407422721386), ('mlp.6', 'head.16.20.k', 0.004757486749440432), ('mlp.0', 'head.1.15.k', -0.004494336899369955), ('mlp.0', 'mlp.5', -0.004315529949963093), ('mlp.0', 'mlp.4', -0.004094669129699469), ('head.16.20', 'mlp.18', 0.004019735846668482), ('head.0.30', 'mlp.0', 0.0039080469869077206), ('mlp.0', 'mlp.6', -0.0038432518485933542), ('mlp.8', 'mlp.9', 0.00376768596470356), ('head.16.20', 'head.21.9.v', 0.003543522208929062), ('mlp.9', 'mlp.11', 0.003415714716538787), ('mlp.6', 'head.21.

In [11]:
import pickle
with open('localizations/eap/eap_sports/1000_graph.pkl', 'wb') as f:
    pickle.dump(graph, f)

import pickle
with open('localizations/eap/eap_sports/1000_eap_scores.pkl', 'wb') as f:
    pickle.dump(graph.eap_scores, f)